### Retrieve Site and Parse HTML

In [ ]:
# pip install requests
import requests
from bs4 import BeautifulSoup


url = "https://mykbostats.com/players/1"
apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3" 
params = {
    "url": url,
    "apikey": apikey,
    "js_render": "true",
    "premium_proxy": "true",
}
response = requests.get("https://api.zenrows.com/v1/", params=params)

#print(response.content[:500])

soup = BeautifulSoup(response.content, 'html.parser')

### Player Name and Team

In [ ]:
temp = soup.select('title')[0].text

player = temp[:temp.find(' - ')]

print(player)

### Table Headers

In [287]:
# extract table header information
header = soup.select('thead th')

for i in range(len(header)):
  header[i] = header[i].text.strip()

# remove 'Game Stats' data
header = header[:header.index("Date")]

print(header)

['Year', 'Team', 'ERA', 'WHIP', 'W', 'L', 'SV', 'H', 'BSV', 'G', 'GS', 'CG(SHO)', 'QS', 'TBF', 'NP', 'IP', 'R', 'ER', 'H', '2B', '3B', 'HR', 'SO', 'BB', 'IBB', 'HB', 'WP', 'BK']


### Table Contents

In [288]:
rows = soup.select('tbody tr')

r = rows[0]

temp = [r.select_one('.left').text.strip(), r.select_one('nobr').text.strip()]
for i in r.select('td')[2:]:
  temp.append(i.text.strip())


print(temp)

# Need to adjust team name to include full team name

['2022', 'NC', '10.38', '2.19', '0', '0', '0', '0', '0', '6', '0', '0 (0)', '0', '44', '157', '8\xa0⅔', '10', '10', '16', '5', '1', '2', '3', '3', '0', '1', '1', '0']


### Combine Into Dictionary

In [ ]:
data = {player:None} # temp[0] is Year info


dt = {temp[0]:{}}
for d in range(1, len(header)):
  dt[temp[0]][header[d]] = temp[d]

data[player] = dt
print(data)

In [ ]:
# Player Name
t = soup.select('title')[0].text.strip()
player = t[:t.find(' - ')]


# extract table header information
header = soup.select('thead th')
for i in range(len(header)):
  header[i] = header[i].text.strip()

# remove 'Game Stats' data
header = header[:header.index("Date")]


teams = {'Doosan': 'Doosan Bears', 'Hanwha': 'Hanwha Eagles', 'Kia': 'Kia Tigers', 'Kiwoom': 'Kiwoom Heroes', 'KT': 'KT Wiz', 'LG': 'LG Twins', 'Lotte': 'Lotte Giants', 'NC': 'NC Dinos', 'Samsung': 'Samsung Lions', 'SSG': 'SSG Landers'}

# Init Dict
data = {player:{}}

# add row data
rows = soup.select('tbody tr')

for r in rows:
  if (r.select_one('.left').text.strip() == 'Career'):
    break

  temp = [r.select_one('.left').text.strip(), r.select_one('nobr').text.strip()]  
  for i in r.select('td')[2:]:
    temp.append(i.text.strip())
  
  temp[1] = teams[temp[1]]
  temp = {head : temp[i] for i, head in enumerate(header)}
  year = temp.pop('Year')

  data[player].update({
    year : {}
  })

  data[player][year].update(temp)


{'Kang Dong-yeon KBO League Pitching Stats': {'2022': {'Team': 'NC Dinos', 'ERA': '10.38', 'WHIP': '2.19', 'W': '0', 'L': '0', 'SV': '0', 'H': '16', 'BSV': '0', 'G': '6', 'GS': '0', 'CG(SHO)': '0 (0)', 'QS': '0', 'TBF': '44', 'NP': '157', 'IP': '8\xa0⅔', 'R': '10', 'ER': '10', '2B': '5', '3B': '1', 'HR': '2', 'SO': '3', 'BB': '3', 'IBB': '0', 'HB': '1', 'WP': '1', 'BK': '0'}, '2021': {'Team': 'NC Dinos', 'ERA': '7.83', 'WHIP': '1.70', 'W': '3', 'L': '2', 'SV': '0', 'H': '25', 'BSV': '0', 'G': '12', 'GS': '3', 'CG(SHO)': '0 (0)', 'QS': '0', 'TBF': '109', 'NP': '435', 'IP': '23', 'R': '21', 'ER': '20', '2B': '4', '3B': '1', 'HR': '4', 'SO': '14', 'BB': '14', 'IBB': '0', 'HB': '0', 'WP': '3', 'BK': '0'}, '2020': {'Team': 'NC Dinos', 'ERA': '6.00', 'WHIP': '1.58', 'W': '1', 'L': '2', 'SV': '0', 'H': '27', 'BSV': '1', 'G': '22', 'GS': '0', 'CG(SHO)': '0 (0)', 'QS': '0', 'TBF': '106', 'NP': '410', 'IP': '24', 'R': '17', 'ER': '16', '2B': '6', '3B': '0', 'HR': '3', 'SO': '17', 'BB': '11', 'IB

In [313]:
d = {head : temp[i] for i, head in enumerate(header)}

print(d.pop('Year'))
print(d)

2022
{'Team': 'NC', 'ERA': '10.38', 'WHIP': '2.19', 'W': '0', 'L': '0', 'SV': '0', 'H': '16', 'BSV': '0', 'G': '6', 'GS': '0', 'CG(SHO)': '0 (0)', 'QS': '0', 'TBF': '44', 'NP': '157', 'IP': '8\xa0⅔', 'R': '10', 'ER': '10', '2B': '5', '3B': '1', 'HR': '2', 'SO': '3', 'BB': '3', 'IBB': '0', 'HB': '1', 'WP': '1', 'BK': '0'}


{1: 'as'}